# Анализ базы данных сервиса для чтения книг по подписке.

## 1. Цели и задачи исследования.

**Цель** - проанализировать базу данных, содержащую информацию о книгах, издательствах, авторах, а также пользовательские обзоры книг. Анализ  данных поможет сформулировать ценностное предложение для нового продукта.

**Задачи:**

- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, исключив таким образом из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учесть только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

**Описание данных**

*Таблица `books`*

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

*Таблица `authors`*

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

*Таблица `publishers`*

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

*Таблица `ratings`*

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

*Таблица `reviews`*

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## 2. Загрузка и изучение данных датафреймов.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
query1 = '''SELECT * FROM authors LIMIT 5'''
query2 = '''SELECT * FROM publishers LIMIT 5'''
query3 = '''SELECT * FROM ratings LIMIT 5'''
query4 = '''SELECT * FROM reviews LIMIT 5'''

con=engine.connect()

print('Книги')
display(pd.io.sql.read_sql(sql=text(query), con = con))
print('Авторы')
display(pd.io.sql.read_sql(sql=text(query1), con = con))
print('Издательства')
display(pd.io.sql.read_sql(sql=text(query2), con = con))
print('Оценки')
display(pd.io.sql.read_sql(sql=text(query3), con = con))
print('Обзоры')
display(pd.io.sql.read_sql(sql=text(query4), con = con))

Книги


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Авторы


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Издательства


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Оценки


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Обзоры


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## 3. Расчет количества книг, вышедших после 1 января 2000 года.

In [2]:
query5 = '''

SELECT COUNT(num_pages)
FROM books
WHERE CAST(publication_date AS date) >= '2000-01-01'

'''

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query5), con = con))

,count
0,821


**Вывод**

С 1 января 2000 г вышла 821 книга.

## 4. Расчет количества обзоров и средней оценки для каждой книги.

In [3]:
query6 = '''

-- узнаем количество книг в датафрейме reviews
SELECT COUNT(DISTINCT book_id)
      FROM reviews;

 '''     

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query6), con = con))

,count
0,994


In [4]:
query7 = '''

-- узнаем количество книг в датафрейме ratings
SELECT COUNT(DISTINCT book_id)
      FROM ratings;

 '''     

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query7), con = con))

,count
0,1000


In [5]:
query8 = '''

WITH
--первый подзапрос
tb1 AS
     (SELECT book_id,
             COUNT(reviews.username) AS review
      FROM reviews
      GROUP BY book_id),
      
-- второй подзапрос 
tb2 AS 
    (SELECT book_id,
            AVG(ratings.rating) AS avg_rating
    FROM ratings
    GROUP BY book_id)
    
--общий запрос
SELECT tb1.book_id,
       tb1.review,
       tb2.avg_rating   
FROM tb1 FULL OUTER JOIN tb2 ON tb2.book_id = tb1.book_id
ORDER BY tb1.review DESC
LIMIT 10;
 '''   
    

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query8), con = con))

,book_id,review,avg_rating
0,NaN,NaN,4.000000
1,NaN,NaN,4.000000
2,NaN,NaN,4.000000
3,NaN,NaN,3.000000
4,NaN,NaN,5.000000
5,NaN,NaN,3.666667
6,948.0,7.0,3.662500
7,497.0,6.0,4.125000
8,627.0,6.0,3.789474
9,299.0,6.0,4.287500


**Вывод**

Для каждой книги расчитаны количество обзоров и средняя оценка. Всего книг 1000шт. У книг, по которым не проведен ни один обзор, в ячейках столбца 'review' будет пропуск.

## 5. Определение издательства, которое выпустило наибольшее число книг толще 50 страниц.

In [6]:
query9 = '''

 
SELECT publishers.publisher,
       COUNT(books.title) AS book_count
FROM books
LEFT OUTER JOIN publishers ON publishers.publisher_id = books.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY book_count DESC
LIMIT 1         

'''

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query9), con = con))

,publisher,book_count
0,Penguin Books,42


**Вывод**

Издательство Penguin Books выпустило больше всего книг - 42шт.

## 6. Определение автора с самой высокой средней оценкой книг.

(учитывайте только книги с 50 и более оценками)

In [7]:
query10 = '''

WITH
--первый подзапрос средний рейтинг по каждой книге с условием количества оценок книги больше 50
tb3 AS 
    (SELECT book_id,
            AVG(rating) AS avg_rating
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50),
    
    
-- второй подзапрос соединим таблицы books и authors
tb4 AS
    (SELECT books.book_id,
            books.title,
            authors.author      
    FROM books
    LEFT OUTER JOIN authors ON authors.author_id = books.author_id)

-- объединяем подзапросы и считаем средний рейтинг автора по всем книгам
SELECT tb4.author,
       AVG(tb3.avg_rating) AS rating  
FROM tb4 INNER JOIN tb3 ON tb3.book_id = tb4.book_id
GROUP BY tb4.author
ORDER BY AVG(tb3.avg_rating) DESC
LIMIT 1;

'''

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query10), con = con))

,author,rating
0,J.K. Rowling/Mary GrandPré,4.283844


**Вывод**

Самый высокий средний рейтинг по оценке всех книг автора принадлежит J.K. Rowling - 4,28.

## 7. Расчет среднего количества обзоров от пользователей, которые поставили больше 48 оценок.

In [8]:
query11 = '''

WITH
--первый подзапрос средний рейтинг по каждой книге с условием количества оценок книги больше 48
tb5 AS 
(SELECT username
 FROM ratings
 GROUP BY username
 HAVING COUNT(rating) > 48),

-- второй подзапрос расчет среднего количества обзоров по пользователям
tb6 AS
(SELECT COUNT(review_id) AS count_review,
        username
 FROM reviews
 GROUP BY username)

-- объединяем и считаем среднее количество обзоров

SELECT AVG(tb6.count_review)
FROM tb6 
INNER JOIN tb5 ON tb5.username = tb6.username;

'''

con=engine.connect()
display(pd.io.sql.read_sql(sql=text(query11), con = con))

,avg
0,24.0


**Вывод** 

В среднем количество обзоров от пользователей , поставивших больше 48 оценок - 24.

## 8. Общий вывод.

Анализ базы данных позволил получить следующие результаты:

1. Начиная с 2000 года, была выпущена 821 книга.
2. Из 1000 книг максимальная средняя оценка книги 5.0, минимальная - 1.5.
3. В данных присутствуют книги у которых не т ни одного обзора.
4. Максимальное количество книг выпустило издательство Penguin Books - 42шт.
5. Самую высокую среднюю оценку книг получила J.K. Rowling - 4.28.
6. Среднее количество обзоров от пользователей, поставивших больше 48 оценок - 24.